# Семинар 3. Исправление опечаток

In [2]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Я удалили из данных случаи, когда в словах пропущен или вставлен пробел, чтобы было проще сопоставить слова в предложении. 

In [3]:
#os.getcwd()
bad = open('/home/alina/Рабочий стол/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('/home/alina/Рабочий стол/correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [5]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [6]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [7]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [8]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13016983016983016


Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [9]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Тексты из википедии для этого хорошо подходят.

In [10]:
corpus = open('wiki_data.txt', encoding='utf8').read()

Попробуем предсказать ошибку простым заглядыванием в словарь. Если слово не в словаре - оно неправильное.

In [11]:
# создаем словарь
vocab = set(re.findall('\w+', corpus.lower()))


In [12]:
def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

    
    

In [13]:
# для оценки создаем два списка y_true и y_pred
# проходимся по предложениям
# сопоставляем слова с помощью функции align_words
# проходимся по парам слов и
# если слова одинаковые добавляем в y_true 0 
# если слова разные добавляем в y_true 1
# предказываем ошибочность слова из bad списка 
# добавляем предсказание в список y_pred

y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))
    

Оценим качество.

In [14]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/1200px-Precisionrecall.svg.png",
     width=300, height=300)

Метрики можно долго разбирать, но если коротко, то точность - показывает процент правильности предсказаний, а полнота - показывает какую долю всех возможных положительных примеров правильно предсказывает модель. А f1-мера - объединяет точность и полноту в одну метрику.

Хорошая модель должна иметь хорошую точность и полноту, но иногда точность все-таки важнее и лучше иногда подкрутить модель по порогам вероятности, чтобы модель предсказывала либо точно, либо никак. Например, в задаче определения спама лучше иногда прислать пользователю письмо со спамом, чем определять весь спам и иногда захватывать нормальные письма. 

In [15]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred, ))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94      8707
           1       0.59      0.88      0.71      1303

    accuracy                           0.91     10010
   macro avg       0.79      0.89      0.83     10010
weighted avg       0.93      0.91      0.91     10010



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известного алгоритма Питера Норвига.

Ошибки вычисляются также, по словарю. Но ещё мы собираем частотности, чтобы потом выбирать самое вероятное исправление.

In [16]:
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [17]:
WORDS.most_common(10)

[('в', 267296),
 ('и', 147115),
 ('на', 81926),
 ('с', 61681),
 ('года', 43894),
 ('по', 37235),
 ('году', 32197),
 ('из', 29150),
 ('был', 23293),
 ('не', 23228)]

In [18]:
"солнце" in WORDS

True

In [19]:
"апофеоз" in WORDS

True

In [20]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [21]:
P('солнце')

2.4440966240624417e-05

In [22]:
P('апофеоз')

1.939759225446382e-07

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [23]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return deletes

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [24]:
%%time
correction('сонце')

CPU times: user 54 µs, sys: 8 µs, total: 62 µs
Wall time: 69.4 µs


'соне'

In [25]:
%%time
correction('опофеоз')

CPU times: user 215 µs, sys: 31 µs, total: 246 µs
Wall time: 255 µs


'опофеоз'

In [26]:
word = 'сонце'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [27]:
splits[:10]

[('', 'сонце'),
 ('с', 'онце'),
 ('со', 'нце'),
 ('сон', 'це'),
 ('сонц', 'е'),
 ('сонце', '')]

In [28]:
deletes = [L + R[1:] for L, R in splits if R]

In [29]:
deletes[:10]

['онце', 'снце', 'соце', 'соне', 'сонц']

In [34]:
# здесь будет словарь с удалениями из правильных слов
# слово - ключ и значение - список всех вариантов с удалениями, которые мы нагенерили выше
dictionary_with_deletes = {}
for word in WORDS:
    dictionary_with_deletes.setdefault(word,[]).append(edits1(word))
print(dictionary_with_deletes['солнце'])
#transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

[['олнце', 'слнце', 'сонце', 'солце', 'солне', 'солнц']]


In [30]:
#transposes[:10]

In [31]:
#letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
#replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [32]:
#len(replaces)

In [33]:
#inserts = [L + c + R for L, R in splits for c in letters]

In [34]:
#inserts[:10]

In [35]:
#len(inserts)

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [36]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


Получается, что ошибок стало ещё больше.

In [37]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)
#0.7287712287712288
#0.5057559478127398
#0.23785459974732973

0.6801198801198801
0.15349194167306215
0.24107040312392328


Ещё проблема тут в том, что алгоритм медленно работает для длинных слов.

In [38]:
%%time
correction('солнце')

CPU times: user 26 µs, sys: 3 µs, total: 29 µs
Wall time: 36.2 µs


'солнце'

In [39]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 2.46 ms, sys: 0 ns, total: 2.46 ms
Wall time: 2.35 ms


'насмехатьсяаававттававаываываы'

Посмотрим, как исправляются самые частотные ошибки.

In [40]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'сеня'),
 ('вообще', 'вобще', 'обще'),
 ('вообще', 'ваще', 'аще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кате'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'като'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'то')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

Самое известное расстояние редактирования - расстояние Левенштейна. Тут мы не будет разбирать алгоритм, можете почитать [тут](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0), а код на питоне есть [тут](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0)

Ещё есть расстояние Дамерау-Левенштейна - почти то же самое, только разрешена ещё операция перестановки.

Про самого Левенштейна можно почитать вот тут - https://nplus1.ru/material/2017/09/25/vladimir-levenshtein

In [41]:
# в питоне есть библиотеке для нахождения близких строк

In [42]:
from difflib import get_close_matches

In [43]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

CPU times: user 1.69 s, sys: 0 ns, total: 1.69 s
Wall time: 1.68 s


['апофеоз']

Работает тоже не очень быстро.

Есть библиотека textdistance, в которой реализованы многие методы нахождения расстояний.

In [41]:
import textdistance

In [42]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [43]:
%%time
get_closest_match_with_metric('сонце', WORDS, 3, textdistance.hamming)

CPU times: user 7.09 s, sys: 1.21 ms, total: 7.09 s
Wall time: 7.1 s


[('конце', 0.8), ('монце', 0.8), ('сонче', 0.8)]

In [44]:
%%time
get_closest_match_with_metric('опофеоз', WORDS, 5, textdistance.damerau_levenshtein)

CPU times: user 49.9 s, sys: 26.7 ms, total: 49.9 s
Wall time: 49.9 s


[('апофеоз', 0.8571428571428572),
 ('апофеоза', 0.75),
 ('апофеозом', 0.6666666666666667),
 ('апофеты', 0.5714285714285714),
 ('опорной', 0.5714285714285714)]

In [45]:
%%time
get_closest_match_with_metric('кул', WORDS, 5, textdistance.damerau_levenshtein)

CPU times: user 23.3 s, sys: 9.32 ms, total: 23.3 s
Wall time: 23.2 s


[('кул', 1.0), ('акул', 0.75), ('коул', 0.75), ('куль', 0.75), ('кулл', 0.75)]

Можно немного ускорить поиск, сократив количество слов в словаре. Возьмем только те, что встречаются больше 5 раз.

In [46]:
WORDS = Counter(re.findall('\w+', corpus.lower()))
WORDS = {word:count for word, count in WORDS.items() if count > 5}

In [47]:
%%time
get_closest_match_with_metric('кул', WORDS, 5, textdistance.damerau_levenshtein)

CPU times: user 4.48 s, sys: 0 ns, total: 4.48 s
Wall time: 4.47 s


[('кул', 1.0), ('акул', 0.75), ('коул', 0.75), ('куль', 0.75), ('кули', 0.75)]

С большим словарем даже оптимизированные версии будут работать очень долго. Давайте попробуем избавиться от необходимости сравнивать слово со всеми в словаре.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово. Косинусное расстояние между векторами слов не равно расстоянию редактирования, т.к. в нем не учитывается порядок символов. Близкими будут слова, состоящие из одинаковых символов. Но мы можем быть уверены, что близкими точно не будут слова с большим косинсным расстоянием, и можем их не проверять их честной метрикой редактирования.

In [48]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [49]:
WORDS = Counter(re.findall('\w+', corpus.lower()))
vocab = list(dictionary_with_deletes.keys())
id2word = {i:word for i, word in enumerate(vocab)}

# в sklearn 2 основных векторизатора - CountVectorizer, TfidfVectorizer
# первый превращает текст в вектора по частотности слов (текст "один один два" - вектор [2, 1])
# во втором частотности умножаются на idf слова

# в векторайзер тексты подаются списком строк
# он сам сделает предобработку и токенизацию

#т.к. у нас вместо текстов отдельные слова, мы ставим analyzer='char'
# т.е. слова рассматриваются как последовательность символов
# в ngram_range=(1,n) можно задать размер символьных нграммов, которые хотим получить
# мы ставим 1,1 т.к нам нужны отдельные символы
#min_df - это минимальное количество документов (слов в нашем случае), в которых должен встретиться символ, 
# чтобы попасть в словарь
vec = CountVectorizer(analyzer='char', ngram_range=(1,1), min_df=10)
X = vec.fit_transform(vocab)

In [50]:
def get_closest_match_vec(text, X, vec, topn=20):
    # превращаем слово в вектор такой же размерности
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0] #distance - чем больше, тем хуже, а similarity наоборот
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [53]:
%%time
get_closest_match_vec('сонце', X, vec) # это расстояние - чем меньше тем лучше

CPU times: user 135 ms, sys: 3.97 ms, total: 139 ms
Wall time: 135 ms


[('цоссен', 0.05131670194948623),
 ('цоссене', 0.05612019255146106),
 ('концессионное', 0.08649972160886044),
 ('солнце', 0.0871290708247231),
 ('сценой', 0.0871290708247231),
 ('сценок', 0.0871290708247231),
 ('сосновец', 0.09630388588493599),
 ('носе', 0.10557280900008414),
 ('броненосец', 0.10557280900008414),
 ('сенокосцев', 0.10557280900008414),
 ('самоценное', 0.10557280900008414),
 ('сцен', 0.10557280900008414),
 ('онсе', 0.10557280900008414),
 ('бесценного', 0.10557280900008414),
 ('сено', 0.10557280900008414),
 ('броненосце', 0.10557280900008414),
 ('соне', 0.10557280900008414),
 ('лентоносец', 0.10557280900008414),
 ('осен', 0.10557280900008414),
 ('бессоннице', 0.10557280900008414)]

Напишем функцию, которая принимает слово и находит ближайшее к нему в словаре с помощью косисного расстояния и честной метрики редактирования. 

In [54]:
def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    # ваш код здесь
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    sims = Counter()
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup,topn, metric=metric)

    
    return closest

In [56]:
%time
get_closest_hybrid_match('сонце', X, vec)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


[('солнце', 0.8333333333333334),
 ('соне', 0.8),
 ('онсе', 0.6),
 ('сосновец', 0.5),
 ('бессоннице', 0.5)]

Оценим такой метод исправления.

In [55]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], WORDS):
            pred = get_closest_hybrid_match(pair[1], X, vec)[0][0]
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [56]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.846953046953047
0.4259401381427475
0.09004249454461927


Процент неправильно исправленных снизился и теперь общий процент правильных почти равен проценту изначально правильных.

Чтобы улучшить качество исправлений, мы можем попробовать учитывать контекст. В этом нам помогут языковые модели. Мы можем обучить языковую модель на нашем новостном корпусе и потом домножать каждое расстояние на вероятность.

In [57]:
corpus_wiki = [['<start>', '<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]

In [58]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [59]:
unigrams = Counter()
bigrams =  Counter()
trigrams = Counter()

for sentence in corpus_wiki:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence,2))
    trigrams.update(ngrammer(sentence,3))

In [62]:
# оцените качество также как и раньше
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

WORDS = dict(WORDS)
bigrams = dict(bigrams)
trigrams = dict(trigrams)
unigrams = dict(unigrams)
'''Надо наоборот
Сначала проверить можно ли по триграммной, а потом биграммной
Только вам сам prevprev не нужен по отдельности
Нужно проверить что такой триграмм есть в словаре и что биграм из prevprevword и prevword есть в словаре
(Это для триграммной)
Если нет тогда проверить что есть биграм из word и prevword и униграм из prevword
(Это для биграммной)
И ещё вы когда пытаетесь посчитать вероятность триграмма то делите счётчик триграмма на вероятность биграма
А надо на счётчик биграма'''

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    # это же триграмма!!! вот она!!!
    word_pairs = [('<start>', '<start>')] + word_pairs
    pred_sent = []
    for j in range(1, len(word_pairs)):
        
        pred = None
        
        # проверяем, что слова нет в словаре, чтобы не исправлять все слова
        if not predict_mistaken(word_pairs[j][1], WORDS):
            pred = word_pairs[j][1]
            
        
        else:
            # находим кандидатов для исправления
            predicted = get_closest_hybrid_match(word_pairs[j][1], X, vec)
            # print(predicted,'predicted')
            # [('симпатичнее', 0.7857142857142857), ('симпатичен', 0.6428571428571428), 
            # ('опаснейшим', 0.4285714285714286),
            # ('антиимпериалистической', 0.36363636363636365), ('чемпионате', 0.3571428571428571)]
            # берем предыдущее слово для контекста
            prev_word = word_pairs[j-1][0]
            lm_predicted = []
            #print(prev_word) # <start>
            # это предыдущее предыдущего для триграммы
            prev_prev_word = word_pairs[j-2][0]
            #print(prev_prev_word) # camera
            # это биграм
            bigram = ' '.join([prev_prev_word, prev_word])
            # print(bigram) # camera <start>
            # проверяем, можно ли по триграмной модели, эта штука не выполняется из за проверки...
            if bigram in bigrams:
                for word, m in predicted:
                    trigram = ' '.join([prev_prev_word, prev_word, word])
                    if trigram in trigrams:
                        # print(trigram,'trigram') #camera <start> симпатичнее
                        lm_predicted.append((word, (m)*((trigrams[trigram]/bigrams[bigram]))))
                    else:
                        # если у нас нет в модели такого слова, то мы не сможем вероятность посчитать
                        # остается только взять первое по близости
                        # print(prev_word,'prev_word')# <start> prev_word
                        # print(unigrams[prev_word],'hck')
                        if prev_word not in unigrams:
                            pred = predicted[0][0]
                            print(pred,'pred')
                            for word, m in predicted:
                                for word, m in predicted:
                                    bigram = ' '.join([prev_word, word])
                                    if bigram in bigrams:
                                        lm_predicted.append((word, (m)*((bigrams[bigram]/unigrams[prev_word]))))
    
        if lm_predicted:
            pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]

        if pred is None:
            pred = word_pairs[j][1]
        
        if pred == word_pairs[j][0]:
            correct += 1
        else:
            mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
        total += 1
            
        if word_pairs[j][0] == word_pairs[j][1]:
            total_correct += 1
            if word_pairs[j][0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word_pairs[j][0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [63]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8662337662337662
0.00844205679201842
0.005397955667853452


Процент сломанных не изменился, но стало больше правильных исправлений. Возможно, если использовать биграммную модель, то будет лучше?

## Готовые инструменты

Есть несколько готовых опечаточников:  
1) Hunspell - https://pypi.org/project/hunspell/  
2) Jamspell - https://github.com/bakwc/JamSpell#python  
3) Яндекс.Спеллер - https://yandex.ru/dev/speller/ (только через API)


Если вам понадобится в серьезной задаче исправлять опечатки, то начните с них, а не с алгоритма Норвига.